### Sentinel Playground Pulling

In [14]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
from PIL import Image

import random
import skimage
import skimage.feature
import skimage.viewer
import numpy as np

C:\Users\lawso\AppData\Local\Continuum\anaconda3\envs\geospatial\lib\site-packages\skimage\viewer\utils\__init__.py:1: UserWarning: Recommended matplotlib backend is `Agg` for full skimage.viewer functionality.
  from .core import *


In [15]:
# Setting up folders to properly save the images
os.makedirs("Collected_Images/Sentinel", exist_ok = True)
os.makedirs("Collected_Images/Sentinel/Bands", exist_ok = True)
os.makedirs("Collected_Images/Sentinel/Bands/Cropped", exist_ok = True)
os.makedirs("Collected_Images/Sentinel/FRM_pics", exist_ok = True)
os.makedirs("Collected_Images/Sentinel/GAN_pics", exist_ok = True)
os.makedirs("Collected_Images/Sentinel/SAT_pics", exist_ok = True)

In [3]:
# Creating the base url for sentinel hub playground so that 
# the program can create the correct urls to pull images from
base_url = "https://apps.sentinel-hub.com/sentinel-playground/?source=S2&"
lat = "lat=%s&"
lng = "lng=%s&"
zoom = "zoom=%s&"
preset = "preset=%s&"
other_1 = "layers=B01,B02,B03&maxcc=%s&gain=1.0&gamma=1.0&"
other_2 = "time=2019-12-01|2020-06-25&atmFilter=&showDates=false"

full_url = base_url+lat+lng+zoom+preset+other_1+other_2

In [4]:
# Names of the areas that we are going to be pulling from
names = [
         'Nebraska',
         'Illinois',
         'Arizona_Phoenix',
         'Chihuahua_N',
         'Wyoming',
         'Texas',
         'W_Virginia',
         'New_Mexico_Franklin',
         'Colorodo'
        ]

# The different preset bands that are available on the sentinel playground
bands = [
         '1-NATURAL-COLOR',
         '2_COLOR_INFRARED__VEGETATION_',
         '3_FALSE_COLOR__URBAN_',
         '4_AGRICULTURE',
         '5_VEGETATION_INDEX',
         '6_MOISTURE_INDEX',
         '7_GEOLOGY',
         '8_BATHYMETRIC',
         '90_ATMOSPHERIC_PENETRATION',
         '91_SWIR',
         '92_NDWI',
         '93-SWIR-2-11-12'
        ]

In [5]:
# The given latitude coordinates for the areas that we are looking at.
lat = [ 
        41.2025,
        42.9462,
        33.4180,   
        29.6976,  
        41.2576,
        31.6621,
        37.0270,
        32.9981,
        37.8675
      ]

# The given longitude coordinates for the areas that we are looking at.
lng = [
        -95.9213,
        -88.2415,
       -112.1428, 
       -105.6235,
       -108.4046,
       -105.6609,
        -81.5405,
       -106.6600,
       -107.1412
      ]

# The desired maximum cloud coverage for the areas that we are looking at.
cc = [
        3,
        3,
        3,
        3,
        3,
        3,
        3,
        10,
        10
]

# The desired zoom level for the sentinel playground images
z = 11

In [6]:
# This cell is to start getting the screenshots from sentinel hub with the different
# bands.

save = "Collected_Images/Sentinel/Bands/"

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.set_window_size(2200, 1800)

for i in range(0, len(names)):
    for band in bands:
        cur_link = full_url % (lat[i], lng[i], z, band, cc[i])
        driver.get(cur_link)
        time.sleep(10)
        driver.refresh()
        time.sleep(10)
        driver.save_screenshot(save + str(names[i]) + "-" + str(band) + ".png")
driver.close()

[WDM] - Current google-chrome version 83.0.4103
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/83.0.4103.39/chromedriver_win32.zip


[WDM] - Unpack archive C:\Users\lawso\.wdm\drivers\chromedriver\83.0.4103.39\win32\chromedriver.zip


In [8]:
# Here the collected band images are going to be cropped so that they are most
# aligned with the formation/satelite that were collected in the notebook
# "Desert, Mountain, and Urban Dataset" 
save = "Collected_Images/Sentinel/"

l = [402, 532, 770, 492, 620, 802, 746, 462, 464]
r = [642, 512, 274, 552, 424, 242, 298, 582, 580]
t = [ 60,  60,  60,  60,  60,  60,  60,  60,  60]
b = [ 28,  28,  28,  28,  28,  28,  28,  28,  28]

names = [
         'Nebraska',
         'Illinois',
         'Arizona_Phoenix',
         'Chihuahua_N',
         'Wyoming',
         'Texas',
         'W_Virginia',
         'New_Mexico_Franklin',
         'Colorodo'
        ]

for i in range(0,len(names)):
    img = Image.open("Collected_Images/DMU_Dataset/GAN_pics/" + names[i] + "-gan.png")
    img = img.crop((0,0,img.width/2,img.height))
    img.save(save + "SAT_pics/" + names[i] + "_sat.png")

    for j in range(0, len(bands)):
        sentinel_img = Image.open(save + "Bands/"+ names[i]+ "-" + bands[j] + ".png")
        w, h = sentinel_img.size

        offset_w_l = l[i]
        offset_w_r = r[i]
        offset_h_t = t[i]
        offset_h_b = b[i]

        left = 0 + offset_w_l
        top = 0 + offset_h_t
        right = w - offset_w_r
        bottom = h - offset_h_b

        sentinel_img = sentinel_img.crop((left, top, right, bottom))


        sentinel_img.save(save + "Bands/Cropped/" + names[i] + "_" + bands[j] + ".png")

In [9]:
# Next to we will get the formation images from the previous notebook into
# a local folder

for i in range(0,len(names)):
    img = Image.open("Collected_Images/DMU_Dataset/GAN_pics/" + names[i] + "-gan.png")
    img = img.crop((img.width/2, 0, img.width, img.height))
    img.save(save + "FRM_pics/" + names[i] + "_frm.png")

In [10]:
# We want to set up the directories in order to create the folders for the model to train
# from.

main_dirs = [
    '1-Natural_Color',
    '2-Infared',
    '3-False_Color_Urban',
    '4-Agriculture',
    '5-Vegetation_Index',
    '6-Moisture_Index',
    '7-Geology',
    '8-Bathymetric',
    '9-Atmospheric_Penetration',
    '10-SWIR',
    '11-NDWI',
    '12-SWIR'
]

for to_make in main_dirs:
    os.makedirs("Collected_Images/Sentinel/" + to_make, exist_ok=True)
    os.makedirs("Collected_Images/Sentinel/" + to_make + "/Final_GAN", exist_ok=True)
    os.makedirs("Collected_Images/Sentinel/" + to_make + "/test", exist_ok=True)
    os.makedirs("Collected_Images/Sentinel/" + to_make + "/train", exist_ok=True)
    os.makedirs("Collected_Images/Sentinel/" + to_make + "/val", exist_ok=True)

In [11]:
# Lastly save off the cropped band images and the formation images, stitch them
# together and then save it off.
save_stub = "Collected_Images/Sentinel/"
save_end_stub = "/Final_GAN/"
bnd_path = "Collected_Images/Sentinel/Bands/Cropped/"
frm_path = "Collected_Images/Sentinel/FRM_pics/"

for i in range(0, len(bands)):
    cur_save = save_stub + main_dirs[i] + save_end_stub
    for area in names:
        cur_bnd = Image.open(bnd_path + area + "_" + bands[i] + ".png")
        cur_frm = Image.open(frm_path + area + "_frm.png")
        
        jnt = Image.new("RGB", (cur_bnd.width + cur_bnd.width, cur_bnd.height))
        jnt.paste(cur_bnd, (0,0))
        jnt.paste(cur_frm, (cur_bnd.width, 0))
        jnt.save(cur_save + area + "_" + main_dirs[i] + ".png")

The next step is to split all the band images into their own separate folders to then be used by the training program

In [16]:
'''
    This function will take in several inputs and will populate a test, train and val
    folder with images that contain the band image on the left, and the formation image
    on the right.
    :param path: path to take the GAN images out of
    :param out_path: the initial directory to save the images
    :param name: The array of names that contain the names of the areas
    :param pic_name: The name of the bands
    :param num_imgs: Number of images desired
    :param img_size: The size of the images desired
    :param test_p: percentage of test images created
    :param train_p: percentage of training images created
    :param val_p: percentage of validation images created
'''

def split_images(path, out_path, name, pic_name, num_imgs, img_size, test_p, train_p, val_p):
    visited = []

    temp_img = []
    for n in name:
        temp_img.append(n + "_" +pic_name + ".png")

    get_size = Image.open(path + temp_img[0])

    w, h = get_size.size

    org_size = h

    n_vals = [
                int(test_p * num_imgs),   # n_vals[0] = test
                int(val_p * num_imgs),    # n_vals[1] = val
                int(train_p * num_imgs)   # n_vals[2] = train
            ]

    paths = [
                out_path + "test/",
                out_path + "val/",
                out_path + "train/"
            ]

    for count in range(num_imgs): 
        temp = random.randrange(0, len(temp_img))
        src_num = temp
        temp = random.randrange(0, h-img_size)
        x = temp
        temp = random.randrange(0, h-img_size)
        y = temp

        cur = {
            "source":str(name[src_num]),
            "coords":(x,y)
        }

        while(len(visited) != 0 and cur in visited):
            temp = random.randrange(0, len(temp_img))
            src_num = temp        
            temp = random.randrange(0, h-img_size)
            x = temp
            temp = random.randrange(0, h-img_size)
            y = temp

            cur = {
            "source":str(name[src_num]),
            "coords":(x,y)
            }

        visited.append(cur)

        img = Image.open(path + temp_img[src_num])

        left = x
        top = y
        right = left + img_size
        bottom = top + img_size

        lft_img = img.crop((left,
                            top,
                            right,
                            bottom))

        rht_img = img.crop((left+h,
                            top,
                            right+h,
                            bottom))

        joint = Image.new("RGB", (lft_img.width + rht_img.width, lft_img.height))
        joint.paste(lft_img, (0,0))
        joint.paste(rht_img, (lft_img.width, 0))

        r = random.randrange(0,3)
        choice = n_vals[r]

        while(choice == 0):
            r = random.randrange(0,3)
            choice = n_vals[r]

        n_vals[r] -= 1

        joint.save(paths[r] + str(count) + "-" + name[src_num] + "-img.png")

In [17]:
# The names of the areas that we will be looking at
name = [
             'Nebraska',
             'Illinois',
             'Arizona_Phoenix',
             'Chihuahua_N',
             'Wyoming',
             'Texas',
             'W_Virginia',
             'New_Mexico_Franklin',
             'Colorodo'
            ]
# creating all the variables needed for the method
num_imgs = 600
img_size = 256
test_p = 1.0/6.0
train_p = 4.0/6.0
val_p = 1.0/6.0

# A loop going through all the different bands creating training, testing 
# and validation images for each type of band
for i in range(0, len(main_dirs)):
    path = "Collected_Images/Sentinel/" + main_dirs[i] + "/Final_GAN/"
    out_path = "Collected_Images/Sentinel/" + main_dirs[i] + "/"
    pic_name = main_dirs[i]

    split_images(path,
                 out_path,
                 name,
                 pic_name,
                 num_imgs,
                 img_size,
                 test_p,
                 train_p,
                 val_p
                )